In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")


In [3]:
from langchain_groq import ChatGroq

llm=ChatGroq(model='deepseek-r1-distill-llama-70b')

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

c:\Users\naran\anaconda3\envs\qa_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [6]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\naran\anaconda3\envs\qa_venv\lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

<think>
Okay, so the user is asking about Task Decomposition. Let me see what I have in the context. 

From the context, I see that Task Decomposition can be done in a few ways. First, using LLMs with simple prompts like asking for steps or subgoals. Second, using task-specific instructions, like outlining a story for writing a novel. Third, with human input. So, it's a method to break down tasks into smaller parts, which can be handled by different models or steps.

I should explain that task decomposition is about breaking tasks into subtasks, and mention the three methods from the context. Also, note that after decomposition, expert models execute each part and log the results. 

I need to keep the answer concise, three sentences max. So, I'll structure it as a definition, the methods, and the execution part. 

Wait, the context also mentions challenges like planning over time and adjusting when errors occur. Should I include that? Probably not, since the user just asked for what it